In [ ]:
from sklearn import preprocessing
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def pcaAnalysis(df, target, features, palette='coolwarm'):
    """
    Description:
        Perform PCA analysis on a dataframe given target and features. Plots the PCA as a scatterplot
        and shows cumulative explained variance ratio for all components.

    Args:
        df (Dataframe): Dataframe on which PCA is to be performed
        target (str): Column label of target value for PCA
        features (list[str]): List of all features from df to consider for PCA
        palette (str): Color palette for hue used in seaborn plot

    Returns:
        None: This function does not return a value.
    """

    # Separating out the features and target
    x = df.loc[:, features].values

    # Standardizing the features
    x = preprocessing.scale(x, with_std=False)

    #PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2'])
    finalDf = pd.concat([principalDf, df[[target]]], axis = 1)

    #Plotting PCA visually
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        x='PC1',  # Name of the first principal component in principalDf
        y='PC2',  # Name of the second principal component in principalDf
        hue=target,  # Color points based on the target column
        data=finalDf,  # Use finalDf as the data source
        palette=palette,  # Set the color palette
        alpha=0.7  # Set transparency for better visibility
    )

    # Add labels and title
    plt.title('PCA Result Visualization', fontsize=16)  
    plt.xlabel('Principal Component 1', fontsize=14)  
    plt.ylabel('Principal Component 2', fontsize=14)
    plt.legend(title=target, bbox_to_anchor=(1.05, 1), loc='upper left')  # Customize legend
    plt.grid(True)  # Add a grid for easier interpretation
    plt.tight_layout()  # Adjust layout to avoid overlapping elements
    plt.show()  # Display the plot

    # Calculate and plot cumulative explained variance ratio for all components
    # Create new PCA object with all possible components
    n_components = min(len(features), len(df))
    pca_full = PCA(n_components=n_components)
    pca_full.fit(x)
    
    # Calculate cumulative explained variance ratio
    cumulative_variance_ratio = np.cumsum(pca_full.explained_variance_ratio_)
    
    # Create new figure for cumulative variance plot
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_variance_ratio) + 1), 
            cumulative_variance_ratio, 
            'bo-', 
            linewidth=2)
    
    # Add reference line at 95% explained variance
    plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
    
    # Customize the plot
    plt.title('Cumulative Explained Variance Ratio by Number of Components', fontsize=16)
    plt.xlabel('Number of Components', fontsize=14)
    plt.ylabel('Cumulative Explained Variance Ratio', fontsize=14)
    plt.grid(True)
    plt.legend()
    
    # Add percentage labels on y-axis
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    
    plt.tight_layout()
    plt.show()

    return x

In [ ]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score
from matplotlib.colors import Normalize

def kmeansStats(df, features, max_k=25):
    """
    Description:
        Perform k-means clustering on a dataframe, present graphs of both
        inertial and silhouette scores for all possible k (capped at 25).

    Args:
        df (Dataframe): Dataframe on which clustering is to be performed
        features (list[str]): List of all features from df to consider for clustering
        max_k (int): Maximum number of clusters to consider in elbow method

    Returns:
        None: This function does not return a value.
    """
    # Separating out the features
    x = df.loc[:, features].values

    # Standardizing the features
    x = preprocessing.scale(x, with_std=False)

    # Calculate metrics for different k values
    inertias = []
    silhouette_scores = []
    K = range(2, max_k + 1)
    
    #Create model for every possible k
    for k in K:
        # Create and fit model
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(x)
        
        # Calculate metrics
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(x, kmeans.labels_))

    # Plot the curves
    plt.figure(figsize=(15, 5))
    
    # Inertia plot
    plt.subplot(1, 2, 1)
    plt.plot(K, inertias, 'bx-')
    plt.xlabel('k (Number of Clusters)')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    plt.grid(True)
    
    # Silhouette score plot
    plt.subplot(1, 2, 2)
    plt.plot(K, silhouette_scores, 'rx-')
    plt.xlabel('k (Number of Clusters)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score vs Number of Clusters')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()


def kmodel(df, optimal_k, features, palette='coolwarm'):
    """
    Description:
        Perform k-means clustering on a dataframe, determine optimal k using elbow method
        and silhouette scores, and visualize results alongside PCA.

    Args:
        df (Dataframe): Dataframe on which clustering is to be performed
        optimal_k (int): k to use for optimal clustering (see prior method for elbow graph)
        features (list[str]): List of all features from df to consider for clustering
        palette (str): Color palette for visualization

    Returns:
        KMeans: Fitted k-means model with optimal number of clusters
    """
    #Generate and standardize x-values
    x = df.loc[:, features].values
    x = preprocessing.scale(x, with_std=False)

    # Fit final model with optimal k
    final_kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    final_kmeans.fit(x)
    
    # Perform PCA for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(x)
    
    # Create a colormap and normalize object
    cmap = plt.get_cmap(palette, optimal_k)  
    norm = Normalize(vmin=0, vmax=optimal_k - 1)  # Normalize cluster labels to colormap range

    # Create visualization of clusters in PCA space
    plt.figure(figsize=(10, 6))

    # Scatter plot with explicit color assignment
    colors = cmap(norm(final_kmeans.labels_))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], color=colors, alpha=0.7)

    # Add labels to each point
    for i, (a, b) in enumerate(zip(X_pca[:, 0], X_pca[:, 1])):
        plt.text(a, b, f'{1974 + i}', fontsize=6, ha='right', va='bottom', color='black')

    # Titles and labels
    plt.title(f'K-means Clustering Results (k={optimal_k})\nVisualized in PCA Space', fontsize=16)
    plt.xlabel('First Principal Component', fontsize=14)
    plt.ylabel('Second Principal Component', fontsize=14)

    # Create a legend based on cluster labels
    for i in range(optimal_k):
        plt.scatter([], [], color=cmap(norm(i)), label=f'Cluster {i}')
    plt.legend(title='Clusters', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Grid and layout
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Print summary statistics for each cluster
    cluster_df = pd.DataFrame(x, columns=features)
    cluster_df['Cluster'] = final_kmeans.labels_
    
    print("\nCluster Sizes:")
    print(cluster_df['Cluster'].value_counts().sort_index())
    
    print("\nCluster Centers (Standardized Features):")
    cluster_centers_df = pd.DataFrame(
        final_kmeans.cluster_centers_,
        columns=features,
        index=[f"Cluster {i}" for i in range(optimal_k)]
    )
    print(cluster_centers_df)
    
    return final_kmeans

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import numpy as np

def DBSCANStats(df, features):
    """
    Description:
        Perform k nearest neighnors claculation on data to determine the eps value
        to be used when performing a DBSCAN later. Plot the different distances
        as a line graph.

    Args:
        df (Dataframe): Dataframe on which clustering is to be performed
        features (list[str]): List of all features from df to consider for clustering

    Returns:
        None: This function does not return a value.
    """
    # Separate and Standardize features
    x = df.loc[:, features].values
    x = preprocessing.scale(x, with_std=False)

    # Compute the distances to the k-th nearest neighbor (k = min_samples - 1)
    neighbors = NearestNeighbors(n_neighbors=5).fit(x)
    distances, _ = neighbors.kneighbors(x)
    distances = np.sort(distances[:, -1])  # Sort the k-distances

    # Plot the k-distance graph
    plt.plot(distances, 'bx-')
    plt.title("k-distance Graph")
    plt.xlabel("Points sorted by distance")
    plt.ylabel("k-distance")
    plt.show()

def performDBSCAN(df, features, eps, min_samples, palette='coolwarm'):
    """
    Description:
        Perform DBSCAN clustering on a dataframe and visualize results using PCA,
        matching the visualization style of KMeans clustering for easy comparison.

    Args:
        df (Dataframe): Dataframe on which clustering is to be performed
        features (list[str]): List of all features from df to consider for clustering
        eps (float): How close data points must be to each other to be considered part of a cluster
        min_samples (int): Minimum number of data points required to form a distinct cluster
        palette (str): Color palette for visualization

    Returns:
        DBSCAN: Fitted DBSCAN model
    """
    # Generate and standardize x-values
    x = df.loc[:, features].values
    x = preprocessing.scale(x, with_std=False)

    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan.fit(x)
    
    # Get cluster labels and number of clusters (excluding noise points labeled as -1)
    labels = dbscan.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    
    # Perform PCA for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(x)
    
    # Create a colormap and normalize object
    cmap = plt.get_cmap(palette, n_clusters + 1)  # +1 for noise points
    norm = Normalize(vmin=-1, vmax=n_clusters - 1)  # Include -1 for noise points
    
    # Create visualization of clusters in PCA space
    plt.figure(figsize=(10, 6))
    
    # Scatter plot with explicit color assignment
    colors = cmap(norm(labels))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], color=colors, alpha=0.7)
    
    # Add labels to each point
    for i, (a, b) in enumerate(zip(X_pca[:, 0], X_pca[:, 1])):
        plt.text(a, b, f'{1974 + i}', fontsize=6, ha='right', va='bottom', color='black')
    
    # Titles and labels
    plt.title(f'DBSCAN Clustering Results (eps={eps}, min_samples={min_samples})\nVisualized in PCA Space', 
              fontsize=16)
    plt.xlabel('First Principal Component', fontsize=14)
    plt.ylabel('Second Principal Component', fontsize=14)
    
    # Create a legend based on cluster labels
    for i in range(-1, n_clusters):
        label = 'Noise' if i == -1 else f'Cluster {i}'
        plt.scatter([], [], color=cmap(norm(i)), label=label)
    plt.legend(title='Clusters', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Grid and layout
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics for each cluster
    cluster_df = pd.DataFrame(x, columns=features)
    cluster_df['Cluster'] = labels
    
    print("\nCluster Sizes:")
    print(cluster_df['Cluster'].value_counts().sort_index())
    
    # Calculate and print cluster centers (excluding noise points)
    print("\nCluster Centers (Standardized Features):")
    centers_list = []
    for i in range(n_clusters):
        mask = labels == i
        if np.any(mask):  # Only calculate center if cluster has points
            center = np.mean(x[mask], axis=0)
            centers_list.append(center)
    
    cluster_centers_df = pd.DataFrame(
        centers_list,
        columns=features,
        index=[f"Cluster {i}" for i in range(n_clusters)]
    )
    print(cluster_centers_df)
    
    return dbscan

In [ ]:
import pandas as pd

#Get raw data from excel
raw_df = pd.read_excel('historicalppi.xlsx')
clean_df = raw_df.copy()

#Process and clean the raw df
new_header = clean_df.iloc[0]
new_header.iloc[1:] = new_header.iloc[1:].astype(int)
clean_df.columns = new_header
clean_df.drop([0,17,18,19], inplace=True)
clean_df.set_index(clean_df.columns[0], drop=True, inplace=True)

'''Create ItemDF from cleanDF'''
#Reset Index to be based off of the item and fill N/A values
itemIndexdf = clean_df.copy()
itemIndexdf.at['Farm-level eggs',1983] = 4.6
itemIndexdf.at['Farm-level eggs',1984] = 12.4
itemIndexdf.reset_index(inplace=True)

'''Create yearDF from cleanDF'''
#Set Producer Price Index item as columns rather than rows
yearIndexdf = clean_df.copy()
yearIndexdf = yearIndexdf.T
yearIndexdf.reset_index(inplace=True)

#Change Header Row
header = list(clean_df.columns.copy())
header[0] = 'Year'
header[1] = 'Unprocessed foodstuffs and feedstuffs'
yearIndexdf.columns = header

#Enforce integer typing for year
yearIndexdf['Year'] = yearIndexdf['Year'].astype(int)

#Need to fill farm-level eggs for 1983 (6.6) & 1984 (14.4)
#.at has to use index and header vals .iat uses only integers
yearIndexdf.at[9,'Farm-level eggs'] = 4.6
yearIndexdf.at[10,'Farm-level eggs'] = 12.4

'''Describe the data using statistics'''
display(yearIndexdf.describe())
display(itemIndexdf.describe())

'''Initial Graph of the data'''
# Graphing of line plots for each item on the same graph with different colors
plt.figure(figsize=(10, 6))  # Create a single figure
for i in range(1, len(yearIndexdf.columns)):  
    sns.lineplot(data=yearIndexdf, x='Year', y=yearIndexdf.columns[i], label=yearIndexdf.columns[i])

plt.title("Features vs. Time")
plt.legend(title="Features", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel("Year")
plt.ylabel("Value")
plt.show() 

In [ ]:
'''PCA Analysis of both DF'''
#PCA analysis for (Year vs Producer Price Index item) and (Producer Price Index item vs Year) respectively
display(yearIndexdf)
pcaAnalysis(yearIndexdf,'Year',list(yearIndexdf.columns.copy())[1:])
display(itemIndexdf)
pcaAnalysis(itemIndexdf,'Producer Price Index item',list(itemIndexdf.columns.copy())[1:])

In [ ]:
'''KMeans Clustering of Year Indexed DF'''
features = list(yearIndexdf.columns.copy())[1:]
kmeansStats(yearIndexdf,features=features)
#Optimal should be k=4 or k=5 according to inertia and silhouette score plots
myKMEANSmodel = kmodel(yearIndexdf, features=features, optimal_k=4)

In [ ]:
'''DBSCAN Clustering of Year Indexed DF'''
DBSCANStats(yearIndexdf,features=features) #kdistance of 80
myDBSCANmodel = performDBSCAN(yearIndexdf,features=features,eps=80,min_samples=4)